In [3]:
import rasterio as rio
import pandas as pd
import geopandas as gpd
from pathlib import Path
from datetime import datetime
import json
import os

In [188]:
s2_image_path = r"C:\Users\Kostas\Downloads\sentinel2_images_mean_2019-04-01_to_2019-05-01-0000000000-0000000000.tif" #OG one
s2_image_path = r"C:\Users\Kostas\Downloads\sentinel2_images_mean_2019-04-01_to_2019-05-01-0000000000-0000006912.tif"
envelopes_gdf = gpd.read_file(r"C:\Users\Kostas\Desktop\GIMA\Module_7\Data\PEP725\After_2016_sent_from_PEP725\pep725_outputs\PEP725_envelopes.geojson")

In [189]:
with rio.open(s2_image_path) as src:
    print(src.bounds)
    print(src.crs)
    print(src.count)
    src.close

BoundingBox(left=695040.0, bottom=5686440.0, right=921420.0, top=6101160.0)
EPSG:32632
23


In [190]:
envelopes_gdf.set_crs(32632, inplace=True, allow_override=True)
envelopes_gdf.tail(10)

,s_id,lon,lat,alt,alt_dem,gss_id,genus,species,phase_id,year,day,date,Label,geometry
129319,4240,10.33330,49.3833,440,463,10000,perm_grass,None,111,2021,224,2021-08-12,M,"POLYGON ((592769.082 5466921.590, 600769.082 5..."
129320,21516,10.86670,52.5667,75,64,10000,perm_grass,None,131,2021,225,2021-08-13,M,"POLYGON ((622519.644 5821708.065, 630519.644 5..."
129321,26751,7.69580,50.5622,290,276,10000,perm_grass,None,111,2021,225,2021-08-13,M,"POLYGON ((403627.815 5597953.720, 411627.815 5..."
129322,4240,10.33330,49.3833,440,463,10000,perm_grass,None,131,2021,228,2021-08-16,M,"POLYGON ((592769.082 5466921.590, 600769.082 5..."
129323,1963,9.00000,49.9167,150,168,10000,perm_grass,None,131,2021,231,2021-08-19,M,"POLYGON ((496000.000 5525369.095, 504000.000 5..."
129324,5456,13.75000,50.7333,875,864,10000,perm_grass,None,131,2021,244,2021-09-01,M,"POLYGON ((831136.405 5626933.921, 839136.405 5..."
129325,961,7.50000,52.7667,30,33,10000,perm_grass,None,111,2021,257,2021-09-14,M,"POLYGON ((394795.749 5843373.189, 402795.749 5..."
129326,20595,8.50000,49.4667,95,86,10000,perm_grass,None,131,2021,258,2021-09-15,M,"POLYGON ((459771.720 5475458.856, 467771.720 5..."
129327,19312,8.36667,48.6000,490,622,10000,perm_grass,None,131,2021,285,2021-10-12,M,"POLYGON ((449306.954 5379184.811, 457306.954 5..."
129328,8197,10.98330,48.2333,525,504,10000,perm_grass,None,111,2021,289,2021-10-16,M,"POLYGON ((643273.716 5340132.606, 651273.716 5..."


In [7]:
# Separate the gdfs by year
#envelopes_gdf_2019 = envelopes_gdf[envelopes_gdf['year'] == 2019]
#envelopes_gdf_2020 = envelopes_gdf[envelopes_gdf['year'] == 2020]

In [8]:
#envelopes_gdf_2019.head()

## 1. Temporal filter

Now that everything is loaded the temporal filter should be applied

In [191]:
# Function to extract the dates from the filename of GEE S2 images

def imageNamingGEEfiles(raster_path):
    # Example file name: sentinel2_images_mean_2019-07-01_to_2019-08-01-0000006912-0000006912.tif
    string_parts = raster_path.split("_")
    start_date = string_parts[3]
    token = string_parts[5]
    token_string_parts = token.split("-")
    end_date = token_string_parts[0] + "-" + token_string_parts[1] + "-" + token_string_parts[2]
    # Save the month and year to variables
    s2month = datetime.strptime(start_date, '%Y-%m-%d').month
    s2year = datetime.strptime(start_date, '%Y-%m-%d').year
    return start_date, end_date, s2month, s2year

a, b, c, d = imageNamingGEEfiles("sentinel2_images_mean_2019-07-01_to_2019-08-01-0000006912-0000006912.tif")
print(a, b, c, d)

2019-07-01 2019-08-01 7 2019


In [10]:
# Pathlib stuff: Path methods: anchor, parent, name, stem, suffixes
# This can be used for easier extraction of dates from the filename
print("anchor: ", Path(s2_image_path).anchor)
print("parent: ", Path(s2_image_path).parent)
print("name: ", Path(s2_image_path).name)
print("stem: ", Path(s2_image_path).stem)
print("suffixes: ", Path(s2_image_path).suffixes)
print("Normal print: ", s2_image_path)


anchor:  C:\
parent:  C:\Users\Kostas\Downloads
name:  sentinel2_images_mean_2019-04-01_to_2019-05-01-0000000000-0000000000.tif
stem:  sentinel2_images_mean_2019-04-01_to_2019-05-01-0000000000-0000000000
suffixes:  ['.tif']
Normal print:  C:\Users\Kostas\Downloads\sentinel2_images_mean_2019-04-01_to_2019-05-01-0000000000-0000000000.tif


In [192]:
# Get the start and end dates of the image from its name
s2_image_start_date, s2_image_end_date, s2month, s2year = imageNamingGEEfiles(Path(s2_image_path).name)
print(s2_image_start_date, s2_image_end_date, s2month, s2year)

2019-04-01 2019-05-01 4 2019


__________

Datetime operations in order to do date comparisons and find all the dates that are represented in an image

In [193]:
# Converting the date column to datetime data type

envelopes_gdf['date'] = pd.to_datetime(envelopes_gdf['date'], format='%Y-%m-%d').dt.date

In [194]:
# Converting the outputs to datetime.date dtype
s2_image_start_date = datetime.strptime(s2_image_start_date, '%Y-%m-%d').date()
s2_image_end_date = datetime.strptime(s2_image_end_date, '%Y-%m-%d').date()


In [195]:
# Creating a mask to filter the dates that are needed
temporal_mask = (envelopes_gdf.date > s2_image_start_date) & (envelopes_gdf.date < s2_image_end_date)
display(envelopes_gdf.loc[temporal_mask])
s2_image_gdf = envelopes_gdf.loc[temporal_mask]


,s_id,lon,lat,alt,alt_dem,gss_id,genus,species,phase_id,year,day,date,Label,geometry
49410,110,10.48330,53.5000,40,54,1030100,Aesculus,Aesculus hippocastanum,11,2019,92,2019-04-02,DBL,"POLYGON ((594387.807 5924917.608, 602387.807 5..."
49411,148,10.83330,54.2500,55,35,1180100,Betula,Betula pendula,11,2019,92,2019-04-02,DBL,"POLYGON ((615444.341 6008888.454, 623444.341 6..."
49412,299,10.25000,53.5167,35,41,1580100,Fraxinus,Fraxinus excelsior,60,2019,92,2019-04-02,DBL,"POLYGON ((578881.130 5926478.618, 586881.130 5..."
49413,354,8.50000,52.7000,35,41,2290100,Sorbus,Sorbus aucuparia,11,2019,92,2019-04-02,DBL,"POLYGON ((462212.807 5835016.122, 470212.807 5..."
49414,370,8.58333,52.5500,40,40,1050100,Alnus,Alnus glutinosa,11,2019,92,2019-04-02,DBL,"POLYGON ((467747.325 5818295.203, 475747.325 5..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123541,8212,13.88330,51.1333,250,236,10000,perm_grass,None,111,2019,120,2019-04-30,M,"POLYGON ((837588.487 5671994.839, 845588.487 5..."
123542,8227,7.26667,53.0500,3,4,10000,perm_grass,None,111,2019,120,2019-04-30,M,"POLYGON ((379814.909 5875237.296, 387814.909 5..."
123543,20320,8.63333,52.1500,110,82,10000,perm_grass,None,111,2019,120,2019-04-30,M,"POLYGON ((470912.168 5773785.241, 478912.168 5..."
123544,20637,6.53333,51.7333,22,24,10000,perm_grass,None,111,2019,120,2019-04-30,M,"POLYGON ((325669.511 5730255.241, 333669.511 5..."


## Target extraction

In [196]:
def addMonths(gdf):
    # Convert the date to datetime type to work later
    gdf['date'] = pd.to_datetime(gdf['date'])

    # Create a Series with the month (1-12)
    # It finds the month (int 1-12) based on the .month method of the datetime property
    # It achieves that by mapping a lambda function on each element of the date column. Therefore the result is just the month number

    getmonth = gdf['date'].map(lambda x:x.month)
    # Another way
    # test_df = test_df.assign(month=test_df['date'].map(lambda x: x.month))

    # Merge this into the gdf
    gdf = gdf.merge(getmonth, left_index=True, right_index=True)

    # Rename the column
    gdf.rename(columns = {'date_y':'month'}, inplace = True)
    return gdf

In [197]:
envelopes_gdf = addMonths(envelopes_gdf)

In [198]:
# Check for indexing using specific observations
station = 4240
year = 2021
month = 8
envelopes_gdf[(envelopes_gdf['s_id'] == station) & (envelopes_gdf['year'] == year) & (envelopes_gdf['month'] == month)]

,s_id,lon,lat,alt,alt_dem,gss_id,genus,species,phase_id,year,day,date_x,Label,geometry,month
107280,4240,10.3333,49.3833,440,463,2290100,Sorbus,Sorbus aucuparia,286,2021,233,2021-08-21,DBL,"POLYGON ((592769.082 5466921.590, 600769.082 5...",8
107657,4240,10.3333,49.3833,440,463,2230200,Sambucus,Sambucus nigra,286,2021,242,2021-08-30,DBL,"POLYGON ((592769.082 5466921.590, 600769.082 5...",8
129319,4240,10.3333,49.3833,440,463,10000,perm_grass,None,111,2021,224,2021-08-12,M,"POLYGON ((592769.082 5466921.590, 600769.082 5...",8
129322,4240,10.3333,49.3833,440,463,10000,perm_grass,None,131,2021,228,2021-08-16,M,"POLYGON ((592769.082 5466921.590, 600769.082 5...",8


In [18]:
envelopes_gdf.tail()

,s_id,lon,lat,alt,alt_dem,gss_id,genus,species,phase_id,year,day,date_x,Label,geometry,month
129324,5456,13.75000,50.7333,875,864,10000,perm_grass,None,131,2021,244,2021-09-01,M,"POLYGON ((831136.405 5626933.921, 839136.405 5...",9
129325,961,7.50000,52.7667,30,33,10000,perm_grass,None,111,2021,257,2021-09-14,M,"POLYGON ((394795.749 5843373.189, 402795.749 5...",9
129326,20595,8.50000,49.4667,95,86,10000,perm_grass,None,131,2021,258,2021-09-15,M,"POLYGON ((459771.720 5475458.856, 467771.720 5...",9
129327,19312,8.36667,48.6000,490,622,10000,perm_grass,None,131,2021,285,2021-10-12,M,"POLYGON ((449306.954 5379184.811, 457306.954 5...",10
129328,8197,10.98330,48.2333,525,504,10000,perm_grass,None,111,2021,289,2021-10-16,M,"POLYGON ((643273.716 5340132.606, 651273.716 5...",10


In [199]:
# Group observations by s_id and month, and calculate the label with the maximum frequency for each group
freqresults_df = envelopes_gdf.groupby(['s_id', pd.Grouper(key='month'), pd.Grouper(key='year')])\
    .apply(lambda x: pd.Series({'Label': x['Label'].value_counts().index[0],
                                'phase_id': x['phase_id'].value_counts().index[0]}))\
    .reset_index()

# Rename the column with the labels
freqresults_df = freqresults_df.rename(columns={'Label': 'max_label', 'phase_id': 'max_phase_id'})


In [200]:
freqresults_df.head()

,s_id,month,year,max_label,max_phase_id
0,21,2,2017,DBL,60
1,21,2,2020,DBL,60
2,21,2,2021,DBL,60
3,21,3,2017,DBL,60
4,21,3,2018,DBL,60


In [22]:
freqresults_df['max_label'].value_counts()

DBL    37914
M       1744
EC        16
Name: max_label, dtype: int64

In [24]:
freqresults_df['max_phase_id'].value_counts()

60     13869
286    10477
205     6161
11      4952
95      2578
131     1122
182      385
111      113
10        17
Name: max_phase_id, dtype: int64

___________

## Spatial filter

In [201]:
s2_image_gdf = addMonths(s2_image_gdf)

In [202]:
# Convert the envelopes_gdf to a list to work with the functions
s2_image_gdf_list = s2_image_gdf.geometry.tolist()

# This is used to save the indices and then extract the targets directly from the gdf
s2_image_gdf_index_list = s2_image_gdf.index.values.tolist()


In [203]:
s2_image_gdf_index_list[150:160]

[49560, 49561, 49562, 49563, 49564, 49565, 49566, 49567, 49568, 49569]

In [204]:
s2_image_gdf.iloc[150:160]

,s_id,lon,lat,alt,alt_dem,gss_id,genus,species,phase_id,year,day,date_x,Label,geometry,month
49560,5714,12.11670,51.6000,85,72,1030100,Aesculus,Aesculus hippocastanum,11,2019,92,2019-04-02,DBL,"POLYGON ((711840.653 5717153.292, 719840.653 5...",4
49561,5756,12.03330,52.5000,20,32,1010300,Acer,Acer platanoides,60,2019,92,2019-04-02,DBL,"POLYGON ((701884.866 5816977.455, 709884.866 5...",4
49562,5764,12.55000,51.6833,150,158,1580100,Fraxinus,Fraxinus excelsior,60,2019,92,2019-04-02,DBL,"POLYGON ((741389.232 5727782.823, 749389.232 5...",4
49563,5935,11.15000,52.8500,25,39,1010300,Acer,Acer platanoides,60,2019,92,2019-04-02,DBL,"POLYGON ((640777.689 5853750.012, 648777.689 5...",4
49564,6090,11.08330,51.4000,380,400,1030100,Aesculus,Aesculus hippocastanum,11,2019,92,2019-04-02,DBL,"POLYGON ((640916.759 5692367.343, 648916.759 5...",4
49565,6216,10.70000,51.1833,240,222,1580100,Fraxinus,Fraxinus excelsior,60,2019,92,2019-04-02,DBL,"POLYGON ((614813.970 5667582.414, 622813.970 5...",4
49566,6248,10.33330,51.1667,400,445,1010300,Acer,Acer platanoides,60,2019,92,2019-04-02,DBL,"POLYGON ((589219.692 5665207.793, 597219.692 5...",4
49567,6360,11.08330,51.2167,140,126,2290100,Sorbus,Sorbus aucuparia,11,2019,92,2019-04-02,DBL,"POLYGON ((641495.452 5671985.544, 649495.452 5...",4
49568,8133,8.73333,52.5000,40,31,1180100,Betula,Betula pendula,60,2019,92,2019-04-02,DBL,"POLYGON ((477897.639 5812685.428, 485897.639 5...",4
49569,8187,11.56670,49.3500,585,575,1510100,Fagus,Fagus sylvatica,11,2019,92,2019-04-02,DBL,"POLYGON ((682406.583 5465533.574, 690406.583 5...",4


In [93]:
output_dir = Path(r'C:\Users\Kostas\Desktop\GIMA\Module_7\Data\filtered_patches_GEE')

In [205]:
"""Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""

def getFeatures(gdf):
        return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [206]:
'''
This function reads the envelope list and a raster, checks if the polygons are fully contained in the raster 
and returns 5 lists, 4 with the boundary coordinates for all the envelopes that are fully contained in the raster 
and one of their indexes from the full_index_list.
'''

def getContainedEnvelopeCoords (raster, envelope_list, full_index_list):
    with rio.open(raster, driver='GTiff') as src:
        raster_extent = src.bounds
        
        # List initialization
        minx_list = []
        miny_list = []
        maxx_list = []
        maxy_list = []
        index_list = []
        for i in range(0, len(envelope_list)):
            poly_extent = envelope_list[i].bounds

            # Check if the polygon is fully inside the raster's extent
            if (poly_extent[0] >= raster_extent[0] and poly_extent[2] <= raster_extent[2] and
                poly_extent[1] >= raster_extent[1] and poly_extent[3] <= raster_extent[3]):
                    minx_list.append(poly_extent[0])
                    miny_list.append(poly_extent[1])
                    maxx_list.append(poly_extent[2])
                    maxy_list.append(poly_extent[3])
                    index_list.append(full_index_list[i])
    return minx_list, miny_list, maxx_list, maxy_list, index_list

In [207]:
'''
This function receives a raster file (.tif) and the boundary coordinates for a polygon. 
It then clips the raster to the extent of the polygon. 
The polygon has to intersect the raster for the operation to be completed
'''

from shapely.geometry import box
from rasterio.mask import mask

def exportImage(raster, output_path, minx, miny, maxx, maxy):
    # open the raster file (Single Band)
    data = rio.open(raster, driver='GTiff')

    # Create a bounding box from the polygon min-max coordinates    
    bbox = box(minx, miny, maxx, maxy)
    # Create a geodataframe with a single polygon so that it can be used with rasterio
    geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs='32632')
    # Transform the geodataframe to a GeoJSON-like object that can be used as an input in the rasterio mask function
    coords = getFeatures(geo)
    #print(coords)
    
    # Mask and crop the raster AOI where polygon overlaps the whole raster
    out_img, out_transform = mask(data, shapes=coords, crop=True)
    # Define resolution and more
    out_profile = data.profile.copy()
    
    out_profile.update({'driver':'GTiff', 'width': out_img.shape[2],'height': out_img.shape[1], 'transform': out_transform})
    
    # Write the extracted raster patch to a file
    with rio.open(output_path, 'w', **out_profile) as dst:
        dst.write(out_img)
    
    # data.close()
    # data = None

In [208]:
minx_list, miny_list, maxx_list, maxy_list, contained_index_list = getContainedEnvelopeCoords(s2_image_path, s2_image_gdf_list, s2_image_gdf_index_list)

In [209]:
# Create a list with the contained s_id's. It works
contained_s_id_list = []
for i in range(0, len(contained_index_list)):
    sid = s2_image_gdf.loc[contained_index_list[i], 's_id']
    contained_s_id_list.append(sid)


## Combination of everything to mine the labels

In [210]:
freqresults_df.head()

,s_id,month,year,max_label,max_phase_id
0,21,2,2017,DBL,60
1,21,2,2020,DBL,60
2,21,2,2021,DBL,60
3,21,3,2017,DBL,60
4,21,3,2018,DBL,60


In [211]:
freqresults_df[freqresults_df['s_id'] == contained_s_id_list[3]]

,s_id,month,year,max_label,max_phase_id
24658,4702,2,2019,DBL,60
24659,4702,2,2020,DBL,60
24660,4702,2,2021,DBL,60
24661,4702,3,2017,DBL,60
24662,4702,3,2018,DBL,60
24663,4702,3,2019,DBL,60
24664,4702,3,2020,M,182
24665,4702,3,2021,DBL,60
24666,4702,4,2017,DBL,11
24667,4702,4,2018,DBL,60


In [212]:
condition = (freqresults_df['year'] == s2year) & (freqresults_df['month'] == s2month) & (freqresults_df['s_id'] == contained_s_id_list[3])
#freqresults_df['s_id'] == contained_s_id_list[3]
label = freqresults_df.loc[condition, 'max_label'].values[0]
phase_id = freqresults_df.loc[condition, 'max_phase_id'].values[0]
print(f'Station with ID {contained_s_id_list[3]}, for the year {s2year} and month {s2month} has the label {label} and phenophase with id {phase_id}')


Station with ID 4702, for the year 2019 and month 4 has the label DBL and phenophase with id 60


In [169]:
# Same as above but iterating over s_ids. Works.
for station in contained_s_id_list:
    condition = (freqresults_df['year'] == s2year) & (freqresults_df['month'] == s2month) & (freqresults_df['s_id'] == station)
    label = freqresults_df.loc[condition, 'max_label'].values[0]
    phase_id = freqresults_df.loc[condition, 'max_phase_id'].values[0]
    print(f'Station with ID {station}, for the year {s2year} and month {s2month} has the label {label} and phenophase with id {phase_id}')


Station with ID 110, for the year 2019 and month 4 has the label DBL and phenophase with id 11
Station with ID 148, for the year 2019 and month 4 has the label DBL and phenophase with id 11
Station with ID 299, for the year 2019 and month 4 has the label DBL and phenophase with id 11
Station with ID 354, for the year 2019 and month 4 has the label DBL and phenophase with id 11
Station with ID 370, for the year 2019 and month 4 has the label DBL and phenophase with id 11
Station with ID 411, for the year 2019 and month 4 has the label DBL and phenophase with id 11
Station with ID 411, for the year 2019 and month 4 has the label DBL and phenophase with id 11
Station with ID 415, for the year 2019 and month 4 has the label DBL and phenophase with id 11
Station with ID 427, for the year 2019 and month 4 has the label DBL and phenophase with id 60
Station with ID 460, for the year 2019 and month 4 has the label DBL and phenophase with id 60
Station with ID 461, for the year 2019 and month 4

KeyboardInterrupt: 

In [70]:
freqresults_df.head()

,s_id,month,year,max_label,max_phase_id
0,21,2,2017,DBL,60
1,21,2,2020,DBL,60
2,21,2,2021,DBL,60
3,21,3,2017,DBL,60
4,21,3,2018,DBL,60


In [213]:
# Create lists of the maximum frequency labels and s_ids to organise the outputs to folders
unique_labels = freqresults_df['max_label'].unique().tolist()
unique_phase_ids = freqresults_df['max_phase_id'].unique().tolist()

# Create folder from one of the lists. Change unique_phase_ids with unique_labels depending on what you want.
for folder in unique_phase_ids:
    p = Path(output_dir) / str(folder)
    path_exists = Path.exists(p)
    if path_exists:
        print(f'Folder {folder} already exists, skipping...')
    else:
        print(f'Folder {folder} does not exist, creating it...')
        p.mkdir(parents=True, exist_ok=True)

Folder 60 already exists, skipping...
Folder 11 already exists, skipping...
Folder 131 already exists, skipping...
Folder 286 already exists, skipping...
Folder 205 already exists, skipping...
Folder 95 already exists, skipping...
Folder 182 already exists, skipping...
Folder 111 already exists, skipping...
Folder 10 already exists, skipping...


In [214]:
# Debugging to find out why it saves everything as DBL

print("Creating patches for the image: ", Path(s2_image_path).name)

# Iterating over each envelope in the gdf
# Reminder, minx_list, contained_index_list and contained_s_id_list have the same length with the same sequence.
for i in range(0, len(minx_list)):
    # Condition to extract the max_label and max_phase_id from the freqresults_df, based on the s_id of the contained envelope
    condition = (freqresults_df['year'] == s2year) & (freqresults_df['month'] == s2month) & (freqresults_df['s_id'] == contained_s_id_list[i])
    # Get the index and station id for the station with the index in the i-th position
    station_id = contained_s_id_list[i]
    index = contained_index_list[i]

    # Get the maximum frequency label and phase_id for the current envelope based on the month and year
    label = freqresults_df.loc[condition, 'max_label'].values[0]
    phase_id = freqresults_df.loc[condition, 'max_phase_id'].values[0]
    
    output_folder = str(phase_id)
    # Include the aforementioned information in the image name
    output_name = os.path.join(os.path.join(output_dir, output_folder), Path(s2_image_path).stem + f'index_{index}_station_{station_id}_label_{label}_phase_id_{phase_id}.tif')
    print(f"\t Patch {i+1} out of {len(minx_list) + 1}")

    # Export the patches
    exportImage(s2_image_path, output_name, minx_list[i], miny_list[i], maxx_list[i], maxy_list[i])
print('Patch creation completed!') 



    

Creating patches for the image:  sentinel2_images_mean_2019-04-01_to_2019-05-01-0000000000-0000006912.tif
	 Patch 1 out of 1074
	 Patch 2 out of 1074
	 Patch 3 out of 1074
	 Patch 4 out of 1074
	 Patch 5 out of 1074
	 Patch 6 out of 1074
	 Patch 7 out of 1074
	 Patch 8 out of 1074
	 Patch 9 out of 1074
	 Patch 10 out of 1074
	 Patch 11 out of 1074
	 Patch 12 out of 1074
	 Patch 13 out of 1074
	 Patch 14 out of 1074
	 Patch 15 out of 1074
	 Patch 16 out of 1074
	 Patch 17 out of 1074
	 Patch 18 out of 1074
	 Patch 19 out of 1074
	 Patch 20 out of 1074
	 Patch 21 out of 1074
	 Patch 22 out of 1074
	 Patch 23 out of 1074
	 Patch 24 out of 1074
	 Patch 25 out of 1074
	 Patch 26 out of 1074
	 Patch 27 out of 1074
	 Patch 28 out of 1074
	 Patch 29 out of 1074
	 Patch 30 out of 1074
	 Patch 31 out of 1074
	 Patch 32 out of 1074
	 Patch 33 out of 1074
	 Patch 34 out of 1074
	 Patch 35 out of 1074
	 Patch 36 out of 1074
	 Patch 37 out of 1074
	 Patch 38 out of 1074
	 Patch 39 out of 1074
	 Patc

In [187]:
# Check what should be the output values from the freqresults_df of the PEP725 database depending on year and month
for i in [2017, 2018, 2019, 2020, 2021]:
    print(f'Checking for year {i}')
    for j in range(1,13):
        condition = (freqresults_df['year'] == i) & (freqresults_df['month'] == j) & (freqresults_df['s_id'].isin(contained_s_id_list))
        test_df = freqresults_df[condition]
        print('Month:', j, ' ',test_df.max_phase_id.unique(), test_df.max_label.unique())
    print(' ')

Checking for year 2017
Month: 1   [182] ['M']
Month: 2   [ 60 182] ['DBL' 'M']
Month: 3   [ 60  11 182] ['DBL' 'M']
Month: 4   [11 60] ['DBL']
Month: 5   [ 60  11 111] ['DBL' 'M']
Month: 6   [131  60 111 286] ['M' 'DBL']
Month: 7   [286 131  60] ['DBL' 'M']
Month: 8   [286] ['DBL']
Month: 9   [205 286] ['DBL']
Month: 10   [ 95 205 286] ['DBL']
Month: 11   [ 95 205] ['DBL']
Month: 12   [95] ['DBL']
 
Checking for year 2018
Month: 1   [182  60 286] ['M' 'DBL']
Month: 2   [ 60 182] ['DBL' 'M']
Month: 3   [ 60 182  11] ['DBL' 'M']
Month: 4   [60 11] ['DBL']
Month: 5   [ 60  11 111] ['DBL' 'M']
Month: 6   [131  60 286 111] ['M' 'DBL']
Month: 7   [286 111] ['DBL' 'M']
Month: 8   [286 205] ['DBL']
Month: 9   [205 286  95] ['DBL']
Month: 10   [205  95 286] ['DBL']
Month: 11   [205  95] ['DBL']
Month: 12   [95] ['DBL']
 
Checking for year 2019
Month: 1   [182  60] ['M' 'DBL']
Month: 2   [ 60 182] ['DBL' 'M']
Month: 3   [ 60  11 182] ['DBL' 'M']
Month: 4   [11 60] ['DBL']
Month: 5   [ 60  11 111